In [2]:
from BA_Optimization_ML.Optimization import splitcurve_1 as sp
from BA_Optimization_ML.Optimization import Polynomial as pl
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from scipy.integrate import quad
from scipy.optimize import newton
import math



drawing = False # true if mouse is pressed
mode = False # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1
listy=[]
listx=[]

steps=100

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global ix,iy,drawing,mode
    global listx, listy
    
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if mode == True:
                cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-1)
            else:
                cv2.circle(img,(x,y),5,(0,0,255),-1)
                listx.append(x)
                listy.append(-y)            
                
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if mode == True:
            cv2.rectangle(img,(ix,iy),(x,y),(0,255,0),-1)
        else:
            cv2.circle(img,(x,y),5,(0,0,255),-1)
            

In [3]:

def createImg():
    global listy,listx,img,pic
    listy=[]
    listx=[]

    img = np.zeros((512,512,3), np.uint8)
    pic=cv2.namedWindow('image')
    cv2.setMouseCallback('image',draw_circle)
    tempdrawing=drawing
    start=True
    while(start or drawing):
        cv2.imshow('image',img)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('m'):
            mode = not mode
        elif k == 27:
            break

        if drawing:
            start=False

In [ ]:
#cv2.destroyAllWindows

In [4]:

def prepCoor(pos):
    arrayPos=np.array(pos)
    arrayPos=arrayPos-arrayPos[0]
    return arrayPos.reshape(-1,1)


# Regression Model

In [5]:
def getcoef(lis,deg=5):
    sol=[]
    axes=["x","y"]
    for count,lis in enumerate(lis):
        X=np.linspace(0,1,len(lis)).reshape(-1,1)
        y=prepCoor(lis)
        polmodel = make_pipeline(PolynomialFeatures(deg),LinearRegression(normalize=True))
        polmodel.fit(X, y)
        
        score=polmodel.score(X,y)
        print('Accuracy List',axes[count]," :",score)
        sol.append(np.squeeze(polmodel.steps[1][1].coef_))
    return sol

# Polymomoal Class


In [6]:
class Polynomial:
    
    def __init__(self, coefficients):
        """ input: coefficients are in the form a_n, ...a_1, a_0 
        """
        self.coefficients =  coefficients # tuple is turned into a list
     
    def __repr__(self):
        """
        method to return the canonical string representation 
        of a polynomial.
   
        """
        return "Polynomial" + str(self.coefficients)
            
    def __call__(self, x):    
        res = 0
        for index, coeff in enumerate(self.coefficients):
            #print(index,coeff)
            res = res + coeff * x** index
        return res 

# Creat & Plote Curve

In [7]:

class curve: 
    global step
    def __init__ (self,fx,fy,fx_dot=0,fy_dot=0):
        self.fx
        self.fx=fx
        self.fy=fy
        self.fx_dot=fx_dot
        self.fy_dot=fy_dot
        
        
    def fx(t,self):
        return self.fx(t)
    def fy(t,self):
        return self.fy(t)
    def fx_dot(t,self):
        return self.fx_dot(t)
    def fy_dot(t,self):
        return self.fy_dot(t)
    def pxy(self):
        return self.pxy
    
    
    def createList(self):
        global steps
        Fx= np.array([self.fx(i) for i in np.linspace(0,1,steps)])
        Fy= np.array([self.fy(i) for i in np.linspace(0,1,steps)])
        
        self.pxy=sp.split(steps,Fx,Fy)
        #pxy=pxy[:int(steps*0.8)]
        return self.pxy 
        
    '''
    def __repr__(self):
        steps=100;
        time=np.linspace(0,1,steps);
        plt.clf
        plt.plot(self.fx(time[0]),self.fy(time[0]),"g*",markersize=10)
        plt.plot(self.fx(time[-1]),self.fy(time[-1]),"ro",markersize=10)
        plt.plot(self.fx(time),self.fy(time),marker="o",markersize=3)
        plt.legend(["Start","End","Curve"])
        plt.title("Kurve")
        plt.show()
        return 
    '''
def set_Curve(coeff_X,coeff_Y):
    fx=Polynomial(coeff_X)
    fy=Polynomial(coeff_Y)
    
    #Integrate
    coeff_X_dot=[c*i for i,c in enumerate(coeff_X)]
    coeff_X_dot=coeff_X_dot[1:]
    
    coeff_Y_dot=[c*i for i,c in enumerate(coeff_Y)]
    coeff_Y_dot=coeff_Y_dot[1:]
    
    fx_dot=Polynomial(coeff_X_dot)
    fy_dot=Polynomial(coeff_Y_dot)
    
    return curve(fx,fy,fx_dot,fy_dot)
'''
def slit(N,fx,fy,fx_dot,fy_dot):
    
    def integr(x,l=0):
        return math.sqrt(fx_dot(x)**2+fy_dot(x)**2)

    eval_n=np.linspace(0,1,N)
    eval_n=np.delete(eval_n,0)
    
    
    length_total=quad(integr,0,1)
    length_seg=np.linspace(0,length_total[0],N)
    length_seg=np.delete(length_seg,0)
    def equ(x,l):
        erg=quad(integr,0,x)-l
        return erg[0]
    
    eval_n= [newton(equ,0.1,fprime=integr,args=(l, ),maxiter=100,tol=1.48e-05) for l in length_seg]
    print(eval_n)
    
    pxy= np.array([[fx(i),fy(i)] for i in eval_n])
    print(pxy)
    return pxy
'''    
    
    
def plotcurve(curve):
    global steps;
    time=np.linspace(0,1,steps);
    plt.clf
    pxy=curve.createList().T
    print(pxy[0][:],pxy[1])
    
    fig, axs = plt.subplots(1,2)
    axs[0].plot(curve.fx(time[0]),curve.fy(time[0]),"g*",markersize=10)
    axs[0].plot(curve.fx(time[-1]),curve.fy(time[-1]),"ro",markersize=10)
    axs[0].plot(curve.fx(time),curve.fy(time),marker="o",markersize=3)
    axs[0].set_title("Kurve without equal Spacing")
    axs[0].legend(["Start","End","Curve"])
    axs[1].plot(pxy[0][0],pxy[1][0],"g*",markersize=10)
    axs[1].plot(pxy[0][-1],pxy[1][-1],"ro",markersize=10)
    axs[1].plot(pxy[0][:],pxy[1][:],marker="o",markersize=3)
    axs[1].set_title("Kurve with equal Spacing")
    axs[1].legend(["Start","End","Curve"])
    plt.show()
    
#c=set_Curve([1,2,3],[1,1,1,1])

# Display Polynomial View

In [8]:
def drawTrack(curve=None,showPlot=True):
    if (curve is None):
        createImg()
        coef=getcoef([listx,listy],20)
        print("\nCoefficient X:\n",coef[0].T,"\n--------\nCoefficient Y:\n",coef[1])
        c=set_Curve(list(coef[0]),list(coef[1]))
        if showPlot:
            plotcurve(c)
    else:
        if showPlot:
            c=curve
            plotcurve(c)
    return c

In [ ]:
save=drawTrack()

#cv2.imshow("image",img)

In [ ]:
drawTrack(save)